# Additional Terms


- word embeddings
- BLUE score
- autoencoder
- GLUE benchmark
- MultiNLI accuracy
- SQuAD 
- 

# Autoencoder

An autoencoder is a special case of the encoder-decoder architecture in which the input is also provided as the output. In this case the encoder creates an intermediary representation of the data that the decoder then learns how to translate back into the original input space. 

The classical use case of the autoencoder is dimensionality reduction, compression, and noise reduction. The key expectation of a performant autoencoder is that is it able to represent the input sequence in a more compact format.

More recently, the intermediary representation of data has been thought of as a way to identify or extract features. Assuming that the compressed data holds a "more pure" representation of useful information, from this we can reverse engineer which features are relevant and irrelevant. I.e. the original features that were reduced can be removed from the feature set.